In [1]:
import pandas as pd

In [3]:
data_fof = pd.read_csv('/mgpfs/home/sarmantara/dmmapper/data/L0100N1504_FoF.csv', skiprows=28)
data_subhalo = pd.read_csv('/mgpfs/home/sarmantara/dmmapper/data/L0100N1504_Subhalo.csv', skiprows=37)  
data_subhalo['GalaxyID'] = data_subhalo['GalaxyID'].astype(str)
data_fof['GroupID'] = data_subhalo['GalaxyID'].astype(str)

data_subhalo.to_feather('/mgpfs/home/sarmantara/dmmapper/data/L0100N1504_Subhalo.feather')
data_fof.to_feather('/mgpfs/home/sarmantara/dmmapper/data/L0100N1504_FoF.feather')

/tmp/ipykernel_436946/3252503702.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data_fof = pd.read_csv('/mgpfs/home/sarmantara/dmmapper/data/L0100N1504_FoF.csv', skiprows=28)
/tmp/ipykernel_436946/3252503702.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data_subhalo = pd.read_csv('/mgpfs/home/sarmantara/dmmapper/data/L0100N1504_Subhalo.csv', skiprows=37)


In [ ]:
data_fof = pd.read_feather('/mgpfs/home/sarmantara/dmmapper/data/L0100N1504_FoF.feather')  
data_fof

In [ ]:
data_subhalo = pd.read_feather('/mgpfs/home/sarmantara/dmmapper/data/L0100N1504_Subhalo.feather')  
data_subhalo

In [ ]:
same_features = set(data_fof.columns).intersection(set(data_subhalo.columns))
same_features

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
fof_unique = data_fof.drop(columns=same_features)
fof_unique.head()

In [ ]:
subhalo_unique = data_subhalo.drop(columns=same_features)
subhalo_unique.head()

In [3]:
data_1 = pd.read_feather('/mgpfs/home/sarmantara/dmmapper/data/merge_L0025N0376.feather')
data_2 = pd.read_feather('/mgpfs/home/sarmantara/dmmapper/data/merge_L0025N0752.feather')  
data_3 = pd.read_feather('/mgpfs/home/sarmantara/dmmapper/data/merge_L0100N1504.feather')

print("Data 1:", data_1.columns)
print(data_1.info())
print("\nData 2:", data_2.columns)
print(data_2.info())
print("\nData 3:", data_3.columns)
print(data_3.info())

Data 1: Index(['GroupID', 'SnapNum', 'Redshift', 'RandomNumber', 'GroupMass',
       'GroupCentreOfPotential_x', 'GroupCentreOfPotential_y',
       'GroupCentreOfPotential_z', 'NumOfSubhalos', 'Group_M_Crit200',
       'Group_R_Crit200', 'Group_M_Mean200', 'Group_R_Mean200',
       'Group_M_TopHat200', 'Group_R_TopHat200', 'Group_M_Crit500',
       'Group_R_Crit500', 'Group_M_Mean500', 'Group_R_Mean500',
       'Group_M_Crit2500', 'Group_R_Crit2500', 'Group_R_Mean2500',
       'Group_M_Mean2500', 'GalaxyID', 'DescendantID', 'LastProgID',
       'TopLeafID', 'GroupNumber', 'SubGroupNumber', 'CentreOfMass_x',
       'CentreOfMass_y', 'CentreOfMass_z', 'CentreOfPotential_x',
       'CentreOfPotential_y', 'CentreOfPotential_z', 'Velocity_x',
       'Velocity_y', 'Velocity_z', 'KineticEnergy', 'MechanicalEnergy',
       'TotalEnergy', 'Vmax', 'VmaxRadius', 'Mass', 'MassType_DM',
       'HalfMassProjRad_DM', 'HalfMassRad_DM', 'nodeIndex', 'PosInFile',
       'FileNum', 'Image_ID'],
      dty

In [ ]:
print("Columns with all NaNs:", [col for col in numeric_cols if self.data[col].isna().all()])

In [ ]:
"""
Optimized PyTorch Training Code
- Improved data loading
- Enhanced model architecture
- Optimized training loop
"""

################ MODEL OPTIMIZATION ################

class DMHaloMatterRNN(tcn.Module):
    def __init__(self, 
                input_size=None, 
                hidden_size=None, 
                output_size=None, 
                num_layers=2, 
                dropout_rate=0.2,
                activation='gelu', 
                bidirectional=False, 
                feature_dim=None,
                use_layer_norm=False, 
                return_hidden=False):
        super().__init__()
        self.input_size = input_size if input_size is not None else 64
        self.hidden_size = hidden_size if hidden_size is not None else 128
        self.output_size = output_size if output_size is not None else 1
        self.dropout_rate = dropout_rate
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        self.feature_dim = feature_dim or min(self.hidden_size, self.input_size)
        self.use_layer_norm = use_layer_norm
        self.return_hidden = return_hidden
        
        # Get activation function
        self.activation = self.__get_activation_layer(activation)
        
        # Pre-compute bidirectional factor
        self.output_factor = 2 if self.bidirectional else 1
        
        # Streamlined feature extraction
        self.feature_layer = tcn.Linear(self.input_size, self.feature_dim)
        
        # Optional batch normalization - don't create if not needed
        self.batch_norm = tcn.BatchNorm1d(self.feature_dim)
        
        # Simplified output projection
        intermediate_size = max(4, self.hidden_size // 2)
        self.output_proj1 = tcn.Linear(self.hidden_size * self.output_factor, intermediate_size)
        self.output_proj2 = tcn.Linear(intermediate_size, self.output_size)
        
        # Optional layer normalization
        if self.use_layer_norm:
            self.norm_after_rnn = tcn.LayerNorm(self.hidden_size * self.output_factor)
    
    def __get_activation_layer(self, activation):
        activation_name = activation.lower()
        
        # Simplified activation map with most common activations
        activation_map = {
            'relu': tcn.ReLU(inplace=True),
            'leaky_relu': tcn.LeakyReLU(inplace=True),
            'gelu': tcn.GELU(),
            'swish': tcn.SiLU(inplace=True),
            'silu': tcn.SiLU(inplace=True),
        }
        
        return activation_map.get(activation_name, tcn.GELU())
    
    def forward(self, x):
        """Forward pass (not implemented in base class)."""
        raise NotImplementedError("Forward is Not Implemented")
    
    @tc.jit.ignore
    def process_input(self, x):
        """Optimized input processing"""
        # Handle 2D (single step) vs 3D (sequence) input
        is_sequence = x.dim() == 3
        
        if is_sequence:
            # Batch processing for sequences
            batch_size, seq_len, _ = x.shape
            x = x.reshape(-1, self.input_size)
            x = self.feature_layer(x)
            x = self.activation(x)
            
            # Apply batch norm efficiently
            x = self.batch_norm(x)
            
            # Return to sequence form
            return x.reshape(batch_size, seq_len, self.feature_dim)
        else:
            # Process single step
            x = self.feature_layer(x)
            x = self.activation(x)
            x = self.batch_norm(x)
            return x.unsqueeze(1)  # Add sequence dimension
    
    def apply_weight_init(self):
        """Optimized weight initialization"""
        for m in self.modules():
            if isinstance(m, tcn.Linear):
                tcn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    tcn.init.zeros_(m.bias)
            elif isinstance(m, tcn.GRU):
                for name, param in m.named_parameters():
                    if 'weight_ih' in name:
                        tcn.init.xavier_uniform_(param)
                    elif 'weight_hh' in name:
                        tcn.init.orthogonal_(param)
                    elif 'bias' in name:
                        tcn.init.zeros_(param)


class GRUDMHaloMapper(DMHaloMatterRNN):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        # Create GRU layer
        self.gru = tcn.GRU(
            self.feature_dim,
            self.hidden_size,
            self.num_layers,
            batch_first=True,
            dropout=self.dropout_rate if self.num_layers > 1 else 0,
            bidirectional=self.bidirectional
        )
        
        # Apply optimized weight initialization
        self.apply_weight_init()
        
        # Enable cudnn benchmarking for optimal kernels
        if tc.cuda.is_available():
            tc.backends.cudnn.benchmark = True

    def forward(self, x):
        # Process input
        x = self.process_input(x)
        
        # Run through GRU
        features, h_n = self.gru(x)
        
        # Get only the last output efficiently
        features = features[:, -1]
        
        # Apply layer normalization if needed
        if self.use_layer_norm:
            features = self.norm_after_rnn(features)
        
        # Apply functional dropout during training
        if self.training:
            features = F.dropout(features, p=self.dropout_rate, training=True, inplace=True)
        
        # Simplified output projection
        features = self.output_proj1(features)
        features = self.activation(features)
        
        # Final output projection
        output = self.output_proj2(features)
        
        return (output, h_n) if self.return_hidden else output


################ TRAINER OPTIMIZATION ################

class ModelTrainer:
    def __init__(self, model=None, criterion=None, optimizer=None, patience=10, device=None, save_dir=None):
        # Model setup
        self.model = model if model is not None else RNNDMHaloMapper()
        self.device = device if device else ('cuda' if tc.cuda.is_available() else 'cpu')
        self.model = self.model.to(self.device)

        # Criterion and optimizer with better defaults
        self.criterion = criterion or tc.nn.MSELoss()
        self.optimizer = optimizer or tc.optim.AdamW(
            self.model.parameters(), 
            lr=0.001, 
            weight_decay=0.0001,
            eps=1e-8
        )

        # Save and tracking
        self.save_dir = save_dir
        self.metrics_calculator = CustomMetrics()
        self.best_val_loss = float('inf')
        self.patience = patience
        self.patience_counter = 0

        # Training history
        self.history = {
            'train_loss': [],
            'val_loss': [],
            'train_metrics': [],
            'val_metrics': []
        }

        # Configure for performance
        self.__configure_for_performance()

    def __configure_for_performance(self):
        """Configure settings for optimal performance"""
        # Data loaders
        self.train_loader = None
        self.val_loader = None

        # Mixed precision settings
        self.use_amp = self.device == 'cuda'
        if self.use_amp:
            self.scaler = GradScaler()
        
        # Optimize settings for device
        if self.device == 'cuda':
            # Set optimal CUDA settings
            tc.backends.cudnn.benchmark = True
            tc.backends.cudnn.deterministic = False
            # Increase batch size for GPU
            self.log_interval = 25
            self.accumulation_steps = 1
        else:
            # CPU settings
            self.log_interval = 100
            self.accumulation_steps = 1
        
        # Use JIT compilation for CPU if possible
        if self.device == 'cpu' and hasattr(tc, 'compile'):
            try:
                self.model = tc.compile(self.model)
            except:
                pass

    def __optimize_batch_parameters(self):
        """Optimize batch processing parameters"""
        # Auto-tune batching parameters based on the device and model
        if self.device == 'cuda':
            # Find optimal batch size based on model size
            param_count = sum(p.numel() for p in self.model.parameters())
            gpu_mem = tc.cuda.get_device_properties(0).total_memory
            
            if param_count < 1e6:  # Small model
                self.log_interval = 10
            elif param_count < 1e7:  # Medium model
                self.log_interval = 25
            else:  # Large model
                self.log_interval = 50
                
            # Calculate optimal number of workers based on CPU cores
            self.train_loader.num_workers = min(8, os.cpu_count() or 4)
            
            # Set pinned memory for faster data transfer
            if hasattr(self.train_loader, 'pin_memory'):
                self.train_loader.pin_memory = True
                
        else:  # CPU optimizations
            self.log_interval = 100
            if hasattr(self.train_loader, 'num_workers'):
                self.train_loader.num_workers = 0  # Avoid overhead on CPU

    def __train_epoch(self):
        """Optimized training epoch"""
        self.model.train()
        train_loss = 0
        
        # Simplified metrics tracking
        epoch_metrics = {
            'mse': 0, 'rmse': 0, 'mae': 0, 'r2': 0
        }
        
        # Gradient tracking
        grad_norms = []
        
        # Streamlined progress bar
        pbar = tqdm(self.train_loader, desc="Training")
        
        # Track if metrics need update
        metrics_update_counter = 0
        
        for batch_idx, (batch_features, batch_targets) in enumerate(pbar):
            try:
                # Move data to device efficiently
                batch_features = batch_features.to(self.device, non_blocking=True)
                batch_targets = batch_targets.to(self.device, non_blocking=True)
                
                # Mixed precision training
                if self.use_amp:
                    with autocast(device_type=self.device):
                        outputs = self.model(batch_features)
                        if isinstance(outputs, tuple):
                            outputs = outputs[0]
                        loss = self.criterion(outputs, batch_targets)
                        loss = loss / self.accumulation_steps
                    
                    # Scale gradients
                    self.scaler.scale(loss).backward()
                    
                    # Step optimizer at accumulation boundary
                    if (batch_idx + 1) % self.accumulation_steps == 0:
                        # Gradient clipping
                        if batch_idx % self.log_interval == 0:
                            grad_norm = tc.nn.utils.clip_grad_norm_(
                                self.model.parameters(), max_norm=1.0
                            )
                            grad_norms.append(grad_norm.item())
                        
                        # Update weights with scaler
                        self.scaler.step(self.optimizer)
                        self.scaler.update()
                        self.optimizer.zero_grad(set_to_none=True)  # More efficient
                else:
                    # Standard CPU training
                    outputs = self.model(batch_features)
                    if isinstance(outputs, tuple):
                        outputs = outputs[0]
                    loss = self.criterion(outputs, batch_targets)
                    loss = loss / self.accumulation_steps
                    
                    # Backward pass
                    loss.backward()
                    
                    # Step optimizer at accumulation boundary
                    if (batch_idx + 1) % self.accumulation_steps == 0:
                        # Optional gradient clipping
                        if batch_idx % self.log_interval == 0:
                            grad_norm = tc.nn.utils.clip_grad_norm_(
                                self.model.parameters(), max_norm=1.0
                            )
                            grad_norms.append(grad_norm.item())
                        
                        # Update weights
                        self.optimizer.step()
                        self.optimizer.zero_grad(set_to_none=True)
                
                # Accumulate loss for reporting (unscaled)
                train_loss += loss.item() * self.accumulation_steps
                
                # Update metrics less frequently
                if batch_idx % self.log_interval == 0:
                    metrics_update_counter += 1
                    with tc.no_grad():
                        batch_metrics = self.metrics_calculator.calculate_all_metrics(
                            batch_targets.detach(), outputs.detach()
                        )
                        for metric, value in batch_metrics.items():
                            if metric in epoch_metrics:
                                epoch_metrics[metric] += value
                    
                    # Update progress bar
                    if batch_idx > 0:
                        pbar.set_postfix({'loss': f"{loss.item() * self.accumulation_steps:.5f}"})
            
            except RuntimeError as e:
                logger.error(f"Error in training batch {batch_idx}: {e}")
                continue
        
        # Calculate average metrics
        num_eval_steps = max(1, metrics_update_counter)
        avg_loss = train_loss / len(self.train_loader)
        avg_metrics = {metric: value / num_eval_steps for metric, value in epoch_metrics.items()}
        
        return avg_loss, avg_metrics, grad_norms

    def __validate_epoch(self):
        """Optimized validation epoch"""
        self.model.eval()
        val_loss = 0
        val_batches = 0
        
        # Simplified metrics tracking
        epoch_metrics = {
            'mse': 0, 'rmse': 0, 'mae': 0, 'r2': 0
        }
        
        # Metrics update tracking
        metrics_update_counter = 0
        
        # Progress bar
        pbar = tqdm(self.val_loader, desc="Validation")
        
        with tc.no_grad():
            for batch_idx, (batch_features, batch_targets) in enumerate(pbar):
                try:
                    # Move data to device efficiently
                    batch_features = batch_features.to(self.device, non_blocking=True)
                    batch_targets = batch_targets.to(self.device, non_blocking=True)
                    
                    # Forward pass (simpler without autocast for validation)
                    outputs = self.model(batch_features)
                    if isinstance(outputs, tuple):
                        outputs = outputs[0]
                    
                    # Calculate loss
                    loss = self.criterion(outputs, batch_targets)
                    
                    # Track loss
                    val_loss += loss.item()
                    val_batches += 1
                    
                    # Update metrics less frequently
                    if batch_idx % self.log_interval == 0:
                        metrics_update_counter += 1
                        batch_metrics = self.metrics_calculator.calculate_all_metrics(
                            batch_targets, outputs
                        )
                        for metric, value in batch_metrics.items():
                            if metric in epoch_metrics:
                                epoch_metrics[metric] += value
                        
                        # Update progress bar
                        if batch_idx > 0:
                            pbar.set_postfix({'loss': f"{loss.item():.5f}"})
                
                except RuntimeError as e:
                    logger.error(f"Error in validation batch {batch_idx}: {e}")
                    continue
        
        # Calculate average metrics
        num_eval_steps = max(1, metrics_update_counter)
        avg_loss = val_loss / max(1, val_batches)
        avg_metrics = {metric: value / num_eval_steps for metric, value in epoch_metrics.items()}
        
        return avg_loss, avg_metrics

    def __create_save_directory(self, model_name):
        """Create directory for saving model artifacts"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        model_dir = os.path.join(self.save_dir, f"{model_name}_{timestamp}")
        os.makedirs(model_dir, exist_ok=True)
        
        best_model_path = os.path.join(model_dir, f"best_{model_name}.pt")
        train_history_path = os.path.join(model_dir, f"{model_name}_history.npy")
        train_history_file = os.path.join(model_dir, f"{model_name}_history.csv")
        
        return {
            'model_dir': model_dir,
            'best_model_path': best_model_path,
            'train_history_path': train_history_path,
            'train_history_file': train_history_file
        }

    def __save_model_checkpoint(self, path):
        """Save model checkpoint efficiently"""
        # Use torch.save with _use_new_zipfile_serialization=True for faster saving
        tc.save({
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
        }, path, _use_new_zipfile_serialization=True)

    def __save_training_progress(self, epoch, train_loss, val_loss, history_file):
        """Save training progress to CSV file"""
        if epoch == 0:
            with open(history_file, 'w') as f:
                f.write('epoch,train_loss,val_loss\n')
        
        with open(history_file, 'a') as f:
            f.write(f'{epoch},{train_loss},{val_loss}\n')

    def __log_metrics(self, train_metrics, val_metrics):
        """Log metrics efficiently"""
        # Only log essential metrics
        essential_metrics = ['mse', 'rmse', 'r2']
        
        train_line = "  ".join([f"{m}: {train_metrics[m]:.5E}" for m in essential_metrics if m in train_metrics])
        val_line = "  ".join([f"{m}: {val_metrics[m]:.5E}" for m in essential_metrics if m in val_metrics])
        
        logger.info(f"Train: {train_line}")
        logger.info(f"Valid: {val_line}")

    def set_loaders(self, X, y):
        """Set up optimized data loaders"""
        # Create dataset and split
        dataset = TensorDataset(tc.FloatTensor(X), tc.FloatTensor(y))
        train_size = int(0.8 * len(dataset))
        val_size = len(dataset) - train_size
        train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
        
        # Create optimized data loaders
        num_workers = min(8, os.cpu_count() or 4) if self.device == 'cuda' else 0
        pin_memory = self.device == 'cuda'
        
        self.train_loader = DataLoader(
            train_dataset, 
            batch_size=512, 
            shuffle=True,
            num_workers=num_workers,
            pin_memory=pin_memory,
            persistent_workers=num_workers > 0,
            prefetch_factor=2 if num_workers > 0 else None
        )
        
        self.val_loader = DataLoader(
            val_dataset, 
            batch_size=1024,  # Larger batches for validation
            shuffle=False,
            num_workers=num_workers,
            pin_memory=pin_memory,
            persistent_workers=num_workers > 0,
            prefetch_factor=2 if num_workers > 0 else None
        )

    def train(self, num_epochs, model_name, train_loader=None, val_loader=None, X=None, y=None):
        """Optimized training function"""
        # Set up data loaders
        if train_loader and val_loader:
            self.train_loader = train_loader
            self.val_loader = val_loader
        else:
            self.set_loaders(X, y)
        
        # Create save directory
        save_paths = self.__create_save_directory(model_name)
        
        # Initialize learning rate scheduler with better defaults
        scheduler = tc.optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer,
            mode='min',
            factor=0.5,
            patience=3,  # Slightly more patience
            verbose=True
        )
        
        # Training variables
        grad_norms = []
        current_lr = self.optimizer.param_groups[0]['lr']
        
        # Optimize batch parameters
        self.__optimize_batch_parameters()
        
        # Track time and throughput
        start_time = time.time()
        samples_processed = 0
        
        # Training loop
        for epoch in range(num_epochs):
            epoch_start = time.time()
            print(f"\nEpoch [{epoch + 1}/{num_epochs}] | LR: {current_lr:.2e}")
            
            # Training phase
            train_loss, train_metrics, epoch_grad_norms = self.__train_epoch()
            grad_norms.extend(epoch_grad_norms)
            
            # Validation phase
            val_loss, val_metrics = self.__validate_epoch()
            
            # Update learning rate
            scheduler.step(val_loss)
            
            # Check for LR changes
            new_lr = self.optimizer.param_groups[0]['lr']
            if new_lr != current_lr:
                print(f"Learning rate updated: {current_lr:.2e} → {new_lr:.2e}")
                current_lr = new_lr
            
            # Save training progress
            self.__save_training_progress(epoch, train_loss, val_loss, save_paths['train_history_file'])
            
            # Track gradient statistics
            if grad_norms:
                avg_grad_norm = sum(grad_norms[-min(len(grad_norms), len(self.train_loader)):]) / min(len(grad_norms), len(self.train_loader))
                self.history.setdefault('grad_norms', []).append(avg_grad_norm)
            
            # Track samples for throughput calculation
            samples_processed += len(self.train_loader) * self.train_loader.batch_size
            
            # Early stopping check
            if val_loss < self.best_val_loss:
                self.best_val_loss = val_loss
                self.patience_counter = 0
                self.__save_model_checkpoint(save_paths['best_model_path'])
                print(f"New best model saved (Val Loss: {val_loss:.5E})")
            else:
                self.patience_counter += 1
            
            # Check for early stopping
            if self.patience_counter >= self.patience:
                print(f"Early stopping triggered at epoch {epoch + 1}")
                break
            
            # Update history
            self.history['train_loss'].append(train_loss)
            self.history['val_loss'].append(val_loss)
            self.history['train_metrics'].append(train_metrics)
            self.history['val_metrics'].append(val_metrics)
            
            # Calculate throughput
            epoch_time = time.time() - epoch_start
            samples_per_second = len(self.train_loader) * self.train_loader.batch_size / epoch_time
            
            # Print summary
            epoch_summary = (
                f"Train Loss: {train_loss:.5E}, Val Loss: {val_loss:.5E}, "
                f"Throughput: {samples_per_second:.1f} it/s, "
                f"Time: {epoch_time:.2f}s"
            )
            
            print(epoch_summary)
            
            # Log metrics occasionally
            if (epoch + 1) % 5 == 0:
                self.__log_metrics(train_metrics, val_metrics)
        
        # Final performance summary
        total_time = time.time() - start_time
        avg_throughput = samples_processed / total_time
        print(f"Training complete. Total time: {total_time:.2f}s, Avg throughput: {avg_throughput:.1f} it/s")
        
        # Save final history
        np.save(save_paths['train_history_path'], self.history)
        
        return self.history, save_paths['model_dir']


################ HYPERPARAMETER OPTIMIZATION ################

def optimize_hyperparameters():
    """Optimized hyperparameters based on target hardware"""
    # Detect device capabilities
    device = 'cuda' if tc.cuda.is_available() else 'cpu'
    
    # Get system info
    cpu_count = os.cpu_count() or 4
    
    # Optimize based on hardware
    if device == 'cuda':
        # GPU configuration
        gpu_mem = tc.cuda.get_device_properties(0).total_memory
        gpu_mem_gb = gpu_mem / (1024**3)
        
        # Scale batch size based on GPU memory
        if gpu_mem_gb > 16:  # High-end GPU
            batch_size = 2048
            hidden_size = 128
        elif gpu_mem_gb > 8:  # Mid-range GPU
            batch_size = 1024
            hidden_size = 64
        else:  # Entry-level GPU
            batch_size = 512
            hidden_size = 32
        
        # Workers based on CPU cores, with limit for GPU data transfer
        num_workers = min(8, cpu_count)
    else:
        # CPU configuration
        batch_size = 256  # Smaller batches for CPU
        hidden_size = 32  # Smaller model for CPU
        num_workers = 0  # Avoid worker overhead on CPU
    
    # Return optimized config
    return {
        'MODEL_TYPE': 'GRU',
        'BATCH_SIZE': batch_size,
        'HIDDEN_SIZE': hidden_size,
        'NUM_WORKERS': num_workers,
        'DEVICE': device,
        'NUM_LAYERS': 1,  # Simpler model is often faster
        'DROPOUT_RATE': 0.1,  # Lower dropout for speed
        'ACTIVATION': 'swish',
        'USE_LAYER_NORM': False,  # Skip layer norm for speed
        'LEARNING_RATE': 0.001,
        'WEIGHT_DECAY': 0.001,
        'OPTIMIZER': 'adamw'  # AdamW often works better than Adam
    }


################ USAGE EXAMPLE ################

def train_optimized_model():
    """Example of using the optimized code"""
    # Get optimized hyperparameters
    config = optimize_hyperparameters()
    
    # Initialize model with optimized configuration
    model = GRUDMHaloMapper(
        input_size=X_train.shape[1],
        hidden_size=config['HIDDEN_SIZE'],
        output_size=1,
        num_layers=config['NUM_LAYERS'],
        dropout_rate=config['DROPOUT_RATE'],
        activation=config['ACTIVATION'],
        bidirectional=False,
        use_layer_norm=config['USE_LAYER_NORM'],
        return_hidden=True
    ).to(config['DEVICE'])
    
    # Create optimizer - AdamW usually better than Adam
    optimizer = tc.optim.AdamW(
        model.parameters(),
        lr=config['LEARNING_RATE'],
        weight_decay=config['WEIGHT_DECAY'],
        eps=1e-8
    )
    
    # Create trainer
    trainer = ModelTrainer(
        model=model,
        criterion=tc.nn.MSELoss(),
        optimizer=optimizer,
        device=config['DEVICE'],
        save_dir=path_manager.model_path
    )
    
    # Create data loaders with optimized batch size
    train_dataset = TensorDataset(tc.FloatTensor(X_train), tc.FloatTensor(y_train))
    val_dataset = TensorDataset(tc.FloatTensor(X_val), tc.FloatTensor(y_val))
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=config['BATCH_SIZE'],
        shuffle=True,
        num_workers=config['NUM_WORKERS'],
        pin_memory=config['DEVICE'] == 'cuda',
        persistent_workers=config['NUM_WORKERS'] > 0,
        prefetch_factor=2 if config['NUM_WORKERS'] > 0 else None
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=config['BATCH_SIZE'] * 2,  # Larger batches for validation
        shuffle=False,
        num_workers=config['NUM_WORKERS'],
        pin_memory=config['DEVICE'] == 'cuda',
        persistent_workers=config['NUM_WORKERS'] > 0,
        prefetch_factor=2 if config['NUM_WORKERS'] > 0 else None
    )
    
    # Train model
    history, model_dir = trainer.train(
        num_epochs=30,
        model_name=config['MODEL_TYPE'],
        train_loader=train_loader,
        val_loader=val_loader
    )
    
    return model, history, model_dir

In [ ]:
        metrics_update_counter = 0
        
        for batch_idx, (batch_features, batch_targets) in enumerate(pbar):
            try:
                # Move data to device efficiently
                batch_features = batch_features.to(self.device, non_blocking=True)
                batch_targets = batch_targets.to(self.device, non_blocking=True)
                
                # Mixed precision training
                if self.use_amp:
                    with autocast(device_type=self.device):
                        outputs = self.model(batch_features)
                        if isinstance(outputs, tuple):
                            outputs = outputs[0]
                        loss = self.criterion(outputs, batch_targets)
                        loss = loss / self.accumulation_steps
                    
                    # Scale gradients
                    self.scaler.scale(loss).backward()
                    
                    # Step optimizer at accumulation boundary
                    if (batch_idx + 1) % self.accumulation_steps == 0:
                        # Gradient clipping
                        if batch_idx % self.log_interval == 0:
                            grad_norm = tc.nn.utils.clip_grad_norm_(
                                self.model.parameters(), max_norm=1.0
                            )
                            grad_norms.append(grad_norm.item())
                        
                        # Update weights with scaler
                        self.scaler.step(self.optimizer)
                        self.scaler.update()
                        self.optimizer.zero_grad(set_to_none=True)  # More efficient
                else:
                    # Standard CPU training
                    outputs = self.model(batch_features)
                    if isinstance(outputs, tuple):
                        outputs = outputs[0]
                    loss = self.criterion(outputs, batch_targets)
                    loss = loss / self.accumulation_steps
                    
                    # Backward pass
                    loss.backward()
                    
                    # Step optimizer at accumulation boundary
                    if (batch_idx + 1) % self.accumulation_steps == 0:
                        # Optional gradient clipping
                        if batch_idx % self.log_interval == 0:
                            grad_norm = tc.nn.utils.clip_grad_norm_(
                                self.model.parameters(), max_norm=1.0
                            )
                            grad_norms.append(grad_norm.item())
                        
                        # Update weights
                        self.optimizer.step()
                        self.optimizer.zero_grad(set_to_none=True)
                
                # Accumulate loss for reporting (unscaled)
                train_loss += loss.item() * self.accumulation_steps
                
                # Update metrics less frequently
                if batch_idx % self.log_interval == 0:
                    metrics_update_counter += 1
                    with tc.no_grad():
                        batch_metrics = self.metrics_calculator.calculate_all_metrics(
                            batch_targets.detach(), outputs.detach()
                        )
                        for metric, value in batch_metrics.items():
                            if metric in epoch_metrics:
                                epoch_metrics[metric] += value
                    
                    # Update progress bar
                    if batch_idx > 0:
                        pbar.set_postfix({'loss': f"{loss.item() * self.accumulation_steps:.5f}"})
            
            except RuntimeError as e:
                logger.error(f"Error in training batch {batch_idx}: {e}")
                continue
        
        # Calculate average metrics
        num_eval_steps = max(1, metrics_update_counter)
        avg_loss = train_loss / len(self.train_loader)
        avg_metrics = {metric: value / num_eval_steps for metric, value in epoch_metrics.items()}
        
        return avg_loss, avg_metrics, grad_norms

    def __validate_epoch(self):
        """Optimized validation epoch"""
        self.model.eval()
        val_loss = 0
        val_batches = 0
        
        # Simplified metrics tracking
        epoch_metrics = {
            'mse': 0, 'rmse': 0, 'mae': 0, 'r2': 0
        }
        
        # Metrics update tracking
        metrics_update_counter = 0
        
        # Progress bar
        pbar = tqdm(self.val_loader, desc="Validation")
        
        with tc.no_grad():
            for batch_idx, (batch_features, batch_targets) in enumerate(pbar):
                try:
                    # Move data to device efficiently
                    batch_features = batch_features.to(self.device, non_blocking=True)
                    batch_targets = batch_targets.to(self.device, non_blocking=True)
                    
                    # Forward pass (simpler without autocast for validation)
                    outputs = self.model(batch_features)
                    if isinstance(outputs, tuple):
                        outputs = outputs[0]
                    
                    # Calculate loss
                    loss = self.criterion(outputs, batch_targets)
                    
                    # Track loss
                    val_loss += loss.item()
                    val_batches += 1
                    
                    # Update metrics less frequently
                    if batch_idx % self.log_interval == 0:
                        metrics_update_counter += 1
                        batch_metrics = self.metrics_calculator.calculate_all_metrics(
                            batch_targets, outputs
                        )
                        for metric, value in batch_metrics.items():
                            if metric in epoch_metrics:
                                epoch_metrics[metric] += value
                        
                        # Update progress bar
                        if batch_idx > 0:
                            pbar.set_postfix({'loss': f"{loss.item():.5f}"})
                
                except RuntimeError as e:
                    logger.error(f"Error in validation batch {batch_idx}: {e}")
                    continue
        
        # Calculate average metrics
        num_eval_steps = max(1, metrics_update_counter)
        avg_loss = val_loss / max(1, val_batches)
        avg_metrics = {metric: value / num_eval_steps for metric, value in epoch_metrics.items()}
        
        return avg_loss, avg_metrics

    def __create_save_directory(self, model_name):
        """Create directory for saving model artifacts"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        model_dir = os.path.join(self.save_dir, f"{model_name}_{timestamp}")
        os.makedirs(model_dir, exist_ok=True)
        
        best_model_path = os.path.join(model_dir, f"best_{model_name}.pt")
        train_history_path = os.path.join(model_dir, f"{model_name}_history.npy")
        train_history_file = os.path.join(model_dir, f"{model_name}_history.csv")
        
        return {
            'model_dir': model_dir,
            'best_model_path': best_model_path,
            'train_history_path': train_history_path,
            'train_history_file': train_history_file
        }

    def __save_model_checkpoint(self, path):
        """Save model checkpoint efficiently"""
        # Use torch.save with _use_new_zipfile_serialization=True for faster saving
        tc.save({
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
        }, path, _use_new_zipfile_serialization=True)

    def __save_training_progress(self, epoch, train_loss, val_loss, history_file):
        """Save training progress to CSV file"""
        if epoch == 0:
            with open(history_file, 'w') as f:
                f.write('epoch,train_loss,val_loss\n')
        
        with open(history_file, 'a') as f:
            f.write(f'{epoch},{train_loss},{val_loss}\n')

    def __log_metrics(self, train_metrics, val_metrics):
        """Log metrics efficiently"""
        # Only log essential metrics
        essential_metrics = ['mse', 'rmse', 'r2']
        
        train_line = "  ".join([f"{m}: {train_metrics[m]:.5E}" for m in essential_metrics if m in train_metrics])
        val_line = "  ".join([f"{m}: {val_metrics[m]:.5E}" for m in essential_metrics if m in val_metrics])
        
        logger.info(f"Train: {train_line}")
        logger.info(f"Valid: {val_line}")

    def set_loaders(self, X, y):
        """Set up optimized data loaders"""
        # Create dataset and split
        dataset = TensorDataset(tc.FloatTensor(X), tc.FloatTensor(y))
        train_size = int(0.8 * len(dataset))
        val_size = len(dataset) - train_size
        train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
        
        # Create optimized data loaders
        num_workers = min(8, os.cpu_count() or 4) if self.device == 'cuda' else 0
        pin_memory = self.device == 'cuda'
        
        self.train_loader = DataLoader(
            train_dataset, 
            batch_size=512, 
            shuffle=True,
            num_workers=num_workers,
            pin_memory=pin_memory,
            persistent_workers=num_workers > 0,
            prefetch_factor=2 if num_workers > 0 else None
        )
        
        self.val_loader = DataLoader(
            val_dataset, 
            batch_size=1024,  # Larger batches for validation
            shuffle=False,
            num_workers=num_workers,
            pin_memory=pin_memory,
            persistent_workers=num_workers > 0,
            prefetch_factor=2 if num_workers > 0 else None
        )

    def train(self, num_epochs, model_name, train_loader=None, val_loader=None, X=None, y=None):
        """Optimized training function"""
        # Set up data loaders
        if train_loader and val_loader:
            self.train_loader = train_loader
            self.val_loader = val_loader
        else:
            self.set_loaders(X, y)
        
        # Create save directory
        save_paths = self.__create_save_directory(model_name)
        
        # Initialize learning rate scheduler with better defaults
        scheduler = tc.optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer,
            mode='min',
            factor=0.5,
            patience=3,  # Slightly more patience
            verbose=True
        )
        
        # Training variables
        grad_norms = []
        current_lr = self.optimizer.param_groups[0]['lr']
        
        # Optimize batch parameters
        self.__optimize_batch_parameters()
        
        # Track time and throughput
        start_time = time.time()
        samples_processed = 0
        
        # Training loop
        for epoch in range(num_epochs):
            epoch_start = time.time()
            print(f"\nEpoch [{epoch + 1}/{num_epochs}] | LR: {current_lr:.2e}")
            
            # Training phase
            train_loss, train_metrics, epoch_grad_norms = self.__train_epoch()
            grad_norms.extend(epoch_grad_norms)
            
            # Validation phase
            val_loss, val_metrics = self.__validate_epoch()
            
            # Update learning rate
            scheduler.step(val_loss)
            
            # Check for LR changes
            new_lr = self.optimizer.param_groups[0]['lr']
            if new_lr != current_lr:
                print(f"Learning rate updated: {current_lr:.2e} → {new_lr:.2e}")
                current_lr = new_lr
            
            # Save training progress
            self.__save_training_progress(epoch, train_loss, val_loss, save_paths['train_history_file'])
            
            # Track gradient statistics
            if grad_norms:
                avg_grad_norm = sum(grad_norms[-min(len(grad_norms), len(self.train_loader)):]) / min(len(grad_norms), len(self.train_loader))
                self.history.setdefault('grad_norms', []).append(avg_grad_norm)
            
            # Track samples for throughput calculation
            samples_processed += len(self.train_loader) * self.train_loader.batch_size
            
            # Early stopping check
            if val_loss < self.best_val_loss:
                self.best_val_loss = val_loss
                self.patience_counter = 0
                self.__save_model_checkpoint(save_paths['best_model_path'])
                print(f"New best model saved (Val Loss: {val_loss:.5E})")
            else:
                self.patience_counter += 1
            
            # Check for early stopping
            if self.patience_counter >= self.patience:
                print(f"Early stopping triggered at epoch {epoch + 1}")
                break
            
            # Update history
            self.history['train_loss'].append(train_loss)
            self.history['val_loss'].append(val_loss)
            self.history['train_metrics'].append(train_metrics)
            self.history['val_metrics'].append(val_metrics)
            
            # Calculate throughput
            epoch_time = time.time() - epoch_start
            samples_per_second = len(self.train_loader) * self.train_loader.batch_size / epoch_time
            
            # Print summary
            epoch_summary = (
                f"Train Loss: {train_loss:.5E}, Val Loss: {val_loss:.5E}, "
                f"Throughput: {samples_per_second:.1f} it/s, "
                f"Time: {epoch_time:.2f}s"
            )
            
            print(epoch_summary)
            
            # Log metrics occasionally
            if (epoch + 1) % 5 == 0:
                self.__log_metrics(train_metrics, val_metrics)
        
        # Final performance summary
        total_time = time.time() - start_time
        avg_throughput = samples_processed / total_time
        print(f"Training complete. Total time: {total_time:.2f}s, Avg throughput: {avg_throughput:.1f} it/s")
        
        # Save final history
        np.save(save_paths['train_history_path'], self.history)
        
        return self.history, save_paths['model_dir']


In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import os
import logging
from pathlib import Path
import kaleido  # For PDF export

# Set up logging
logger = logging.getLogger(__name__)

class ModelVisualizer:
    def __init__(self, predictions, true_values, spatial_coords, redshifts=None, cosmological_time=None, subhalo_coords=None, save_dir=None):
        """
        Initialize the ModelVisualizer.

        Args:
            predictions (numpy.ndarray): Model predictions.
            true_values (numpy.ndarray): True target values.
            spatial_coords (numpy.ndarray): Spatial coordinates of the data points.
            redshifts (numpy.ndarray, optional): Redshift values for each data point. Defaults to None.
            cosmological_time (numpy.ndarray, optional): Cosmological time values. Defaults to None.
            subhalo_coords (numpy.ndarray, optioanl): Spatial coordinates of subhalos. Defaults to None.
            save_dir (str, optional): Directory to save visualizations. Defaults to None.
        """
        
        self.predictions = np.asarray(predictions).reshape(-1)  # Reshape predictions
        self.true_values = np.asarray(true_values).reshape(-1)  # Reshape true_values
        self.spatial_coords = np.asarray(spatial_coords)
        self.save_dir = save_dir
        
        # Handle length mismatches
        pred_len = len(self.predictions)
        true_len = len(self.true_values)
        spatial_len = len(self.spatial_coords)
        
        if pred_len != true_len or pred_len != spatial_len:
            logger.warning(f"Length mismatch detected: predictions={pred_len}, true_values={true_len}, spatial_coords={spatial_len}")
            
            # Find minimum length to trim all arrays
            min_len = min(pred_len, true_len, spatial_len)
            logger.info(f"Trimming all arrays to minimum length: {min_len}")
            
            # Trim arrays to the same length
            self.predictions = self.predictions[:min_len]
            self.true_values = self.true_values[:min_len]
            self.spatial_coords = self.spatial_coords[:min_len]
            
            logger.info(f"Arrays trimmed successfully. New lengths: predictions={len(self.predictions)}, " 
                       f"true_values={len(self.true_values)}, spatial_coords={len(self.spatial_coords)}")
        
            
        if self.spatial_coords.shape[1] != 3:
            logger.error("Spatial coordinates must be 3-dimensional")
            raise ValueError("Spatial coordinates must be 3-dimensional")
            
        # Process optional arrays, ensuring they match the trimmed length if provided
        if redshifts is not None:
            self.redshifts = np.asarray(redshifts)
            if len(self.redshifts) != len(self.predictions):
                logger.warning(f"Redshifts length ({len(self.redshifts)}) doesn't match predictions ({len(self.predictions)}). Trimming.")
                self.redshifts = self.redshifts[:len(self.predictions)]
        else:
            self.redshifts = None
            
        if cosmological_time is not None:
            self.cosmological_time = np.asarray(cosmological_time)
            if len(self.cosmological_time) != len(self.predictions):
                logger.warning(f"Cosmological time length ({len(self.cosmological_time)}) doesn't match predictions ({len(self.predictions)}). Trimming.")
                self.cosmological_time = self.cosmological_time[:len(self.predictions)]
        else:
            self.cosmological_time = None
            
        if subhalo_coords is not None:
            self.subhalo_coords = np.asarray(subhalo_coords)
            if len(self.subhalo_coords) != len(self.predictions):
                logger.warning(f"Subhalo coordinates length ({len(self.subhalo_coords)}) doesn't match predictions ({len(self.predictions)}). Trimming.")
                self.subhalo_coords = self.subhalo_coords[:len(self.predictions)]
        else:
            self.subhalo_coords = None
        self.save_dir = save_dir
        
        # Calculate min/max radius
        radii = np.linalg.norm(self.spatial_coords, axis=1)
        self.min_radius = np.min(radii[radii > 0])  # Avoid zero radius
        self.max_radius = np.max(radii)
        
        # Calculate spatial limits for consistent visualization
        self.x_limits = [np.min(self.spatial_coords[:, 0]), np.max(self.spatial_coords[:, 0])]
        self.y_limits = [np.min(self.spatial_coords[:, 1]), np.max(self.spatial_coords[:, 1])]
        self.z_limits = [np.min(self.spatial_coords[:, 2]), np.max(self.spatial_coords[:, 2])]
        
        # Expands limits by 10% for better visualization
        self.x_range = self.x_limits[1] - self.x_limits[0]
        self.y_range = self.y_limits[1] - self.y_limits[0]
        self.z_range = self.z_limits[1] - self.z_limits[0]
        
        self.x_limits = [self.x_limits[0] - 0.1 * self.x_range, self.x_limits[1] + 0.1 * self.x_range]
        self.y_limits = [self.y_limits[0] - 0.1 * self.y_range, self.y_limits[1] + 0.1 * self.y_range]
        self.z_limits = [self.z_limits[0] - 0.1 * self.z_range, self.z_limits[1] + 0.1 * self.z_range]

    def _create_save_directory(self, subdir=None):
        """Create save directory if it doesn't exist."""
        if self.save_dir:
            if subdir:
                full_dir = os.path.join(self.save_dir, subdir)
            else:
                full_dir = self.save_dir
            Path(full_dir).mkdir(parents=True, exist_ok=True)
            return full_dir
        return None

    def _get_bw_style(self, index):
        """Get black and white styling for different data series."""
        styles = [
            {'color': 'black', 'dash': 'solid', 'symbol': 'circle'},
            {'color': 'black', 'dash': 'dash', 'symbol': 'square'},
            {'color': 'black', 'dash': 'dot', 'symbol': 'diamond'},
            {'color': 'black', 'dash': 'dashdot', 'symbol': 'triangle-up'},
            {'color': 'gray', 'dash': 'solid', 'symbol': 'x'},
            {'color': 'gray', 'dash': 'dash', 'symbol': 'cross'},
        ]
        return styles[index % len(styles)]

    def plot_prediction_vs_true_2d(self, save_formats=['html', 'pdf']):
        """Create black and white 2D scatter plot of predictions vs true values."""
        fig = go.Figure()
        
        # Perfect prediction line
        min_val = min(np.min(self.predictions), np.min(self.true_values))
        max_val = max(np.max(self.predictions), np.max(self.true_values))
        
        fig.add_trace(go.Scatter(
            x=[min_val, max_val],
            y=[min_val, max_val],
            mode='lines',
            line=dict(color='gray', dash='dash', width=2),
            name='Perfect Prediction',
            showlegend=True
        ))
        
        # Actual data points
        fig.add_trace(go.Scatter(
            x=self.true_values,
            y=self.predictions,
            mode='markers',
            marker=dict(
                color='black',
                symbol='circle',
                size=6,
                line=dict(color='gray', width=1)
            ),
            name='Predictions vs True',
            showlegend=True
        ))
        
        fig.update_layout(
            template='plotly_white',
            title='Model Predictions vs True Values',
            xaxis_title='True Values',
            yaxis_title='Predictions',
            width=800,
            height=600,
            font=dict(color='black'),
            plot_bgcolor='white',
            paper_bgcolor='white'
        )
        
        # Save in different formats
        save_dir = self._create_save_directory('2d_plots')
        if save_dir:
            for fmt in save_formats:
                if fmt == 'html':
                    fig.write_html(os.path.join(save_dir, 'predictions_vs_true.html'))
                elif fmt == 'pdf':
                    fig.write_image(os.path.join(save_dir, 'predictions_vs_true.pdf'))
                elif fmt == 'png':
                    fig.write_image(os.path.join(save_dir, 'predictions_vs_true.png'))
        
        return fig

    def plot_residuals_2d(self, save_formats=['html', 'pdf']):
        """Create black and white 2D residuals plot."""
        residuals = self.predictions - self.true_values
        
        fig = make_subplots(
            rows=2, cols=1,
            subplot_titles=('Residuals vs True Values', 'Residuals Distribution'),
            vertical_spacing=0.12
        )
        
        # Residuals vs true values
        fig.add_trace(
            go.Scatter(
                x=self.true_values,
                y=residuals,
                mode='markers',
                marker=dict(
                    color='black',
                    symbol='circle',
                    size=5,
                    line=dict(color='gray', width=0.5)
                ),
                name='Residuals',
                showlegend=False
            ),
            row=1, col=1
        )
        
        # Zero line
        fig.add_trace(
            go.Scatter(
                x=[np.min(self.true_values), np.max(self.true_values)],
                y=[0, 0],
                mode='lines',
                line=dict(color='gray', dash='dash', width=2),
                name='Zero Line',
                showlegend=False
            ),
            row=1, col=1
        )
        
        # Residuals histogram
        fig.add_trace(
            go.Histogram(
                x=residuals,
                nbinsx=30,
                marker=dict(color='lightgray', line=dict(color='black', width=1)),
                name='Distribution',
                showlegend=False
            ),
            row=2, col=1
        )
        
        fig.update_layout(
            template='plotly_white',
            title='Residuals Analysis',
            height=800,
            font=dict(color='black'),
            plot_bgcolor='white',
            paper_bgcolor='white'
        )
        
        fig.update_xaxes(title_text='True Values', row=1, col=1)
        fig.update_yaxes(title_text='Residuals', row=1, col=1)
        fig.update_xaxes(title_text='Residuals', row=2, col=1)
        fig.update_yaxes(title_text='Frequency', row=2, col=1)
        
        # Save in different formats
        save_dir = self._create_save_directory('2d_plots')
        if save_dir:
            for fmt in save_formats:
                if fmt == 'html':
                    fig.write_html(os.path.join(save_dir, 'residuals_analysis.html'))
                elif fmt == 'pdf':
                    fig.write_image(os.path.join(save_dir, 'residuals_analysis.pdf'))
                elif fmt == 'png':
                    fig.write_image(os.path.join(save_dir, 'residuals_analysis.png'))
        
        return fig

    def plot_spatial_distribution_3d(self, save_formats=['html']):
        """Create 3D spatial distribution with non-overlapping colorbars."""
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=('Predictions', 'True Values', 'Residuals', 'Spatial Coordinates'),
            specs=[[{'type': 'scatter3d'}, {'type': 'scatter3d'}],
                   [{'type': 'scatter3d'}, {'type': 'scatter3d'}]],
            horizontal_spacing=0.05,
            vertical_spacing=0.1
        )
        
        residuals = self.predictions - self.true_values
        
        # Configure colorbar positions to avoid overlap
        colorbar_positions = [
            {'x': 0.23, 'y': 0.85, 'len': 0.35},  # Top left
            {'x': 0.77, 'y': 0.85, 'len': 0.35},  # Top right
            {'x': 0.23, 'y': 0.15, 'len': 0.35},  # Bottom left
            {'x': 0.77, 'y': 0.15, 'len': 0.35}   # Bottom right
        ]
        
        # Predictions
        fig.add_trace(
            go.Scatter3d(
                x=self.spatial_coords[:, 0],
                y=self.spatial_coords[:, 1],
                z=self.spatial_coords[:, 2],
                mode='markers',
                marker=dict(
                    size=4,
                    color=self.predictions,
                    colorscale='Viridis',
                    colorbar=dict(
                        title='Predictions',
                        titleside='right',
                        **colorbar_positions[0]
                    ),
                    showscale=True
                ),
                name='Predictions',
                showlegend=False
            ),
            row=1, col=1
        )
        
        # True Values
        fig.add_trace(
            go.Scatter3d(
                x=self.spatial_coords[:, 0],
                y=self.spatial_coords[:, 1],
                z=self.spatial_coords[:, 2],
                mode='markers',
                marker=dict(
                    size=4,
                    color=self.true_values,
                    colorscale='Plasma',
                    colorbar=dict(
                        title='True Values',
                        titleside='right',
                        **colorbar_positions[1]
                    ),
                    showscale=True
                ),
                name='True Values',
                showlegend=False
            ),
            row=1, col=2
        )
        
        # Residuals
        fig.add_trace(
            go.Scatter3d(
                x=self.spatial_coords[:, 0],
                y=self.spatial_coords[:, 1],
                z=self.spatial_coords[:, 2],
                mode='markers',
                marker=dict(
                    size=4,
                    color=residuals,
                    colorscale='RdBu',
                    colorbar=dict(
                        title='Residuals',
                        titleside='right',
                        **colorbar_positions[2]
                    ),
                    showscale=True
                ),
                name='Residuals',
                showlegend=False
            ),
            row=2, col=1
        )
        
        # Spatial coordinates (colored by distance from origin)
        distances = np.linalg.norm(self.spatial_coords, axis=1)
        fig.add_trace(
            go.Scatter3d(
                x=self.spatial_coords[:, 0],
                y=self.spatial_coords[:, 1],
                z=self.spatial_coords[:, 2],
                mode='markers',
                marker=dict(
                    size=4,
                    color=distances,
                    colorscale='Cividis',
                    colorbar=dict(
                        title='Distance',
                        titleside='right',
                        **colorbar_positions[3]
                    ),
                    showscale=True
                ),
                name='Distance',
                showlegend=False
            ),
            row=2, col=2
        )
        
        fig.update_layout(
            template='plotly_white',
            title='3D Spatial Distribution Analysis',
            height=800,
            width=1200,
            font=dict(color='black'),
            scene=dict(
                xaxis_title='X',
                yaxis_title='Y',
                zaxis_title='Z'
            ),
            scene2=dict(
                xaxis_title='X',
                yaxis_title='Y',
                zaxis_title='Z'
            ),
            scene3=dict(
                xaxis_title='X',
                yaxis_title='Y',
                zaxis_title='Z'
            ),
            scene4=dict(
                xaxis_title='X',
                yaxis_title='Y',
                zaxis_title='Z'
            )
        )
        
        # Save with frame extraction
        save_dir = self._create_save_directory('3d_plots')
        if save_dir:
            for fmt in save_formats:
                if fmt == 'html':
                    fig.write_html(os.path.join(save_dir, 'spatial_distribution_3d.html'))
                elif fmt == 'pdf':
                    fig.write_image(os.path.join(save_dir, 'spatial_distribution_3d.pdf'))
        
        # Create frame snapshots with different zoom levels
        self._create_3d_snapshots(fig, 'spatial_distribution_3d', save_dir)
        
        return fig

    def plot_redshift_evolution(self, save_formats=['html', 'pdf']):
        """Create standalone redshift evolution visualization."""
        if self.redshifts is None:
            logger.warning("No redshift data available for evolution plot")
            return None
        
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=('Predictions vs Redshift', 'True Values vs Redshift', 
                          'Residuals vs Redshift', 'Redshift Distribution'),
            specs=[[{'secondary_y': False}, {'secondary_y': False}],
                   [{'secondary_y': False}, {'secondary_y': False}]]
        )
        
        residuals = self.predictions - self.true_values
        
        # Sort by redshift for line plots
        sort_idx = np.argsort(self.redshifts)
        sorted_z = self.redshifts[sort_idx]
        sorted_pred = self.predictions[sort_idx]
        sorted_true = self.true_values[sort_idx]
        sorted_res = residuals[sort_idx]
        
        # Predictions vs Redshift
        fig.add_trace(
            go.Scatter(
                x=sorted_z,
                y=sorted_pred,
                mode='markers+lines',
                line=dict(color='black', dash='solid', width=2),
                marker=dict(color='black', symbol='circle', size=4),
                name='Predictions',
                showlegend=False
            ),
            row=1, col=1
        )
        
        # True Values vs Redshift
        fig.add_trace(
            go.Scatter(
                x=sorted_z,
                y=sorted_true,
                mode='markers+lines',
                line=dict(color='black', dash='dash', width=2),
                marker=dict(color='black', symbol='square', size=4),
                name='True Values',
                showlegend=False
            ),
            row=1, col=2
        )
        
        # Residuals vs Redshift
        fig.add_trace(
            go.Scatter(
                x=sorted_z,
                y=sorted_res,
                mode='markers',
                marker=dict(color='black', symbol='diamond', size=4),
                name='Residuals',
                showlegend=False
            ),
            row=2, col=1
        )
        
        # Zero line for residuals
        fig.add_trace(
            go.Scatter(
                x=[np.min(sorted_z), np.max(sorted_z)],
                y=[0, 0],
                mode='lines',
                line=dict(color='gray', dash='dash', width=1),
                name='Zero Line',
                showlegend=False
            ),
            row=2, col=1
        )
        
        # Redshift Distribution
        fig.add_trace(
            go.Histogram(
                x=self.redshifts,
                nbinsx=30,
                marker=dict(color='lightgray', line=dict(color='black', width=1)),
                name='Distribution',
                showlegend=False
            ),
            row=2, col=2
        )
        
        fig.update_layout(
            template='plotly_white',
            title='Redshift Evolution Analysis',
            height=800,
            width=1200,
            font=dict(color='black'),
            plot_bgcolor='white',
            paper_bgcolor='white'
        )
        
        # Update axes labels
        fig.update_xaxes(title_text='Redshift', row=1, col=1)
        fig.update_yaxes(title_text='Predictions', row=1, col=1)
        fig.update_xaxes(title_text='Redshift', row=1, col=2)
        fig.update_yaxes(title_text='True Values', row=1, col=2)
        fig.update_xaxes(title_text='Redshift', row=2, col=1)
        fig.update_yaxes(title_text='Residuals', row=2, col=1)
        fig.update_xaxes(title_text='Redshift', row=2, col=2)
        fig.update_yaxes(title_text='Frequency', row=2, col=2)
        
        # Save in different formats
        save_dir = self._create_save_directory('redshift_evolution')
        if save_dir:
            for fmt in save_formats:
                if fmt == 'html':
                    fig.write_html(os.path.join(save_dir, 'redshift_evolution.html'))
                elif fmt == 'pdf':
                    fig.write_image(os.path.join(save_dir, 'redshift_evolution.pdf'))
                elif fmt == 'png':
                    fig.write_image(os.path.join(save_dir, 'redshift_evolution.png'))
        
        return fig

    def _create_3d_snapshots(self, fig, base_name, save_dir):
        """Create snapshots of 3D plots with different camera angles and zoom levels."""
        if not save_dir:
            return
        
        # Create frames directory
        frames_dir = os.path.join(save_dir, f'{base_name}_frames')
        Path(frames_dir).mkdir(parents=True, exist_ok=True)
        
        # Camera configurations
        camera_configs = {
            'default': dict(),
            'zoom_2x': dict(
                eye=dict(x=0.75, y=0.75, z=0.75)
            ),
            'zoom_5x': dict(
                eye=dict(x=0.3, y=0.3, z=0.3)
            ),
            'zoom_10x': dict(
                eye=dict(x=0.15, y=0.15, z=0.15)
            )
        }
        
        # Multiple viewing angles
        angles = {
            'front': dict(eye=dict(x=1.5, y=0, z=0)),
            'side': dict(eye=dict(x=0, y=1.5, z=0)),
            'top': dict(eye=dict(x=0, y=0, z=1.5)),
            'diagonal': dict(eye=dict(x=1.25, y=1.25, z=1.25))
        }
        
        for zoom_name, zoom_config in camera_configs.items():
            for angle_name, angle_config in angles.items():
                # Combine zoom and angle configurations
                camera = {**zoom_config, **angle_config}
                
                # Update figure with camera settings
                fig_copy = fig
                fig_copy.update_layout(
                    scene_camera=camera,
                    scene2_camera=camera,
                    scene3_camera=camera,
                    scene4_camera=camera
                )
                
                # Save as PNG and PDF
                filename_base = f'{base_name}_{zoom_name}_{angle_name}'
                fig_copy.write_image(os.path.join(frames_dir, f'{filename_base}.png'))
                fig_copy.write_image(os.path.join(frames_dir, f'{filename_base}.pdf'))

    def create_comprehensive_report(self):
        """Generate all visualizations and create a comprehensive report."""
        logger.info("Creating comprehensive visualization report...")
        
        # Create all visualizations
        plots = {}
        
        # 2D Plots
        plots['prediction_vs_true'] = self.plot_prediction_vs_true_2d()
        plots['residuals'] = self.plot_residuals_2d()
        
        # 3D Plots
        plots['spatial_3d'] = self.plot_spatial_distribution_3d()
        
        # Redshift evolution (if available)
        if self.redshifts is not None:
            plots['redshift_evolution'] = self.plot_redshift_evolution()
        
        # Create summary statistics
        self._create_summary_statistics()
        
        logger.info("Comprehensive report created successfully!")
        return plots

    def _create_summary_statistics(self):
        """Create a summary statistics file."""
        if not self.save_dir:
            return
        
        stats = {
            'Dataset Size': len(self.predictions),
            'Spatial Dimensions': self.spatial_coords.shape,
            'Prediction Range': [float(np.min(self.predictions)), float(np.max(self.predictions))],
            'True Values Range': [float(np.min(self.true_values)), float(np.max(self.true_values))],
            'Mean Absolute Error': float(np.mean(np.abs(self.predictions - self.true_values))),
            'Root Mean Square Error': float(np.sqrt(np.mean((self.predictions - self.true_values)**2))),
            'R² Score': float(1 - (np.sum((self.true_values - self.predictions)**2) / 
                            np.sum((self.true_values - np.mean(self.true_values))**2))),
            'Spatial Extents': {
                'X': [float(self.x_limits[0]), float(self.x_limits[1])],
                'Y': [float(self.y_limits[0]), float(self.y_limits[1])],
                'Z': [float(self.z_limits[0]), float(self.z_limits[1])]
            }
        }
        
        if self.redshifts is not None:
            stats['Redshift Range'] = [float(np.min(self.redshifts)), float(np.max(self.redshifts))]
        
        # Save statistics as text file
        save_dir = self._create_save_directory()
        if save_dir:
            with open(os.path.join(save_dir, 'summary_statistics.txt'), 'w') as f:
                for key, value in stats.items():
                    f.write(f"{key}: {value}\n")

# Example usage:
if __name__ == "__main__":
    # Example data generation (replace with your actual data)
    n_points = 1000
    spatial_coords = np.random.randn(n_points, 3) * 10
    true_values = np.random.randn(n_points)
    predictions = true_values + np.random.randn(n_points) * 0.1  # Add some noise
    redshifts = np.random.uniform(0, 2, n_points)
    
    # Initialize visualizer
    visualizer = ModelVisualizer(
        predictions=predictions,
        true_values=true_values,
        spatial_coords=spatial_coords,
        redshifts=redshifts,
        save_dir='visualization_output'
    )
    
    # Create comprehensive report
    plots = visualizer.create_comprehensive_report()

In [ ]:
import os
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp
import imageio
from PIL import Image
import io
from directory_manager import PathManager
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots

path_manager = PathManager()
logger = path_manager.get_logger()

class ModelVisualizer:
    def __init__(self, predictions, true_values, spatial_coords, redshifts=None, cosmological_time=None, subhalo_coords=None, save_dir=None):
        """Initialize the ModelVisualizer."""
        # [Keep all your existing __init__ code exactly as is]
        # ... [previous __init__ code remains unchanged] ...
        
        # Add new attributes for visualization settings
        self.bw_line_styles = ['solid', 'dot', 'dash', 'longdash', 'dashdot', 'longdashdot']
        self.bw_symbols = ['circle', 'square', 'diamond', 'cross', 'x', 'triangle-up']
        
    # [Keep all your existing methods until __plot_density_profile]
    
    def __plot_density_profile(self, r_bins: np.ndarray, profile: np.ndarray, title: str, yaxis_title: str) -> go.Figure:
        """Modified to produce black and white plots with different line styles."""
        fig = go.Figure()
        
        fig.add_trace(
            go.Scatter(
                x=r_bins,
                y=profile,
                mode='lines',
                name=r"$\text{" + title + r"}$",
                line=dict(
                    width=2,
                    color='black',
                    dash=self.bw_line_styles[0]  # First line style
                )
            )
        )

        fig.update_layout(
            title=r"$\text{" + title + r"}$",
            xaxis_title=r"$\text{Radius (Mpc)}$",
            yaxis_title=r"$\text{" + yaxis_title + r"}$",
            xaxis=dict(type="log", title_font=dict(size=16)),
            yaxis=dict(type="log", title_font=dict(size=16)),
            template="plotly_white",
            font=dict(family="Arial, sans-serif", size=14),
            plot_bgcolor='white',
            paper_bgcolor='white'
        )

        return fig
    
    def __plot_all_density_profiles(self, r_bins, profiles):
        """Modified for black and white with different line styles."""
        fig = go.Figure()
        
        for i, (name, profile) in enumerate(profiles.items()):
            fig.add_trace(
                go.Scatter(
                    x=r_bins,
                    y=profile,
                    mode='lines',
                    name=f"$\\text{{{name}}}$",
                    line=dict(
                        width=2,
                        color='black',
                        dash=self.bw_line_styles[i % len(self.bw_line_styles)]
                    )
                )
            )
        
        fig.update_layout(
            title=r"$\text{Dark Matter Halos Density Profiles}$",
            xaxis_title=r"$\text{Radius (Mpc)}$",
            yaxis_title=r"$\text{Density (M_{\odot} / Mpc^{3})}$",
            xaxis=dict(type="log", title_font=dict(size=16)),
            yaxis=dict(type="log", title_font=dict(size=16)),
            template="plotly_white",
            font=dict(family="Arial, sans-serif", size=14),
            plot_bgcolor='white',
            paper_bgcolor='white'
        )
        
        return fig
    
    def visualize_dm_distribution(self, a=1.0, b=0.8, c=0.6):
        """Modified to prevent color bar overlap."""
        try:
            # [Previous code until figure creation]
            
            # Create subplots with adjusted spacing for color bars
            fig = sp.make_subplots(
                rows=1,
                cols=3,
                specs=[[{"type": "scatter3d"}, {"type": "scatter3d"}, {"type": "scatter3d"}]],
                subplot_titles=[
                    r"$\text{Dark Matter Distribution}$", 
                    r"$\text{Triaxial Model}$", 
                    r"$\text{Redshift Evolution}$"
                ],
                horizontal_spacing=0.1,  # Increased spacing
                vertical_spacing=0.1
            )
            
            # [Previous trace creation code]
            
            # Modified color bar positions
            fig.update_traces(
                marker_colorbar_x=0.28,  # First color bar
                selector=dict(name=r"$\text{DM Halos}$")
            )
            
            fig.update_traces(
                marker_colorbar_x=0.85,  # Second color bar
                selector=dict(name=r"$\text{Predictions}$")
            )
            
            # [Rest of the previous code]
            
            return fig
        
        except Exception as e:
            logger.error(f"Error in visualization: {e}", exc_info=True)
            return None
    
    def export_animation_frames(self, fig, frames, base_name):
        """Export animation frames as PNG and PDF with different zoom levels."""
        try:
            if not self.save_dir:
                logger.warning("No save directory specified for frame export")
                return
            
            # Create subdirectory for frames
            frame_dir = os.path.join(self.save_dir, f"{base_name}_frames")
            os.makedirs(frame_dir, exist_ok=True)
            
            # Camera views to export
            camera_views = {
                'default': None,
                'zoom2x': dict(eye=dict(x=0.5, y=0.5, z=0.5)),
                'zoom5x': dict(eye=dict(x=0.2, y=0.2, z=0.2)),
                'zoom10x': dict(eye=dict(x=0.1, y=0.1, z=0.1))
            }
            
            for i, frame in enumerate(frames):
                # Create a copy of the figure
                fig_copy = go.Figure(fig)
                
                # Update with frame data
                for j, trace in enumerate(frame.data):
                    if j < len(fig_copy.data):
                        fig_copy.data[j].update(trace)
                
                # Update title if present
                if hasattr(frame, 'name') and frame.name:
                    current_title = fig_copy.layout.title.text
                    if current_title:
                        base_title = current_title.split(" at")[0]
                        fig_copy.update_layout(title=f"{base_title} at {frame.name}")
                
                # Export for each camera view
                for view_name, camera in camera_views.items():
                    if camera:
                        fig_copy.update_layout(scene_camera=camera)
                    
                    # Save as PNG
                    png_path = os.path.join(frame_dir, f"frame_{i:03d}_{view_name}.png")
                    fig_copy.write_image(png_path, width=1200, height=800)
                    
                    # Convert to PDF using matplotlib
                    img = Image.open(png_path)
                    pdf_path = os.path.join(frame_dir, f"frame_{i:03d}_{view_name}.pdf")
                    img.save(pdf_path, "PDF", resolution=100.0)
            
            logger.info(f"Exported frames to {frame_dir}")
            
        except Exception as e:
            logger.error(f"Error exporting frames: {e}", exc_info=True)
    
    def visualize_redshift_evolution_standalone(self):
        """Standalone redshift evolution visualization."""
        try:
            if not hasattr(self, 'redshifts'):
                logger.error("Redshift data not available")
                return None
                
            # Get unique redshifts (high to low)
            unique_redshifts = np.sort(np.unique(self.redshifts))[::-1]
            
            # Create figure with initial frame (highest redshift)
            fig = go.Figure()
            
            # Calculate normalized densities
            denom = np.max(self.predictions) - np.min(self.predictions)
            if denom == 0:
                norm_predictions = np.zeros_like(self.predictions)
            else:
                norm_predictions = (self.predictions - np.min(self.predictions)) / denom
            
            # Add initial trace
            initial_mask = (self.redshifts == unique_redshifts[0])
            fig.add_trace(
                go.Scatter3d(
                    x=self.spatial_coords[initial_mask, 0],
                    y=self.spatial_coords[initial_mask, 1],
                    z=self.spatial_coords[initial_mask, 2],
                    mode='markers',
                    marker=dict(
                        size=3,
                        color=norm_predictions[initial_mask],
                        colorscale='Viridis',
                        opacity=0.8,
                        colorbar=dict(title="Normalized Density", x=0.85)
                    ),
                    name="Dark Matter"
                )
            )
            
            # Create animation frames
            frames = []
            for z in unique_redshifts:
                mask = (self.redshifts == z)
                if not np.any(mask):
                    continue
                    
                frames.append(go.Frame(
                    data=[go.Scatter3d(
                        x=self.spatial_coords[mask, 0],
                        y=self.spatial_coords[mask, 1],
                        z=self.spatial_coords[mask, 2],
                        mode='markers',
                        marker=dict(
                            size=3,
                            color=norm_predictions[mask],
                            colorscale='Viridis',
                            opacity=0.8
                        )
                    )],
                    name=f"z={z:.5E}",
                    traces=[0]
                ))
            
            # Configure layout
            fig.update_layout(
                title="Standalone Redshift Evolution",
                scene=dict(
                    xaxis_title=r"$X \, (\text{Mpc})$",
                    yaxis_title=r"$Y \, (\text{Mpc})$",
                    zaxis_title=r"$Z \, (\text{Mpc})$",
                    aspectmode='cube',
                    xaxis=dict(range=self.x_limits),
                    yaxis=dict(range=self.y_limits),
                    zaxis=dict(range=self.z_limits)
                ),
                updatemenus=[{
                    'type': 'buttons',
                    'buttons': [
                        {
                            'args': [None, {
                                'frame': {'duration': 500, 'redraw': True},
                                'fromcurrent': True,
                                'transition': {'duration': 0}
                            }],
                            'label': 'Play',
                            'method': 'animate'
                        },
                        {
                            'args': [[None], {
                                'frame': {'duration': 0},
                                'mode': 'immediate'
                            }],
                            'label': 'Pause',
                            'method': 'animate'
                        }
                    ],
                    'x': 0.1,
                    'y': 0,
                    'xanchor': 'right',
                    'yanchor': 'top'
                }],
                sliders=[{
                    'active': 0,
                    'currentvalue': {'prefix': 'Redshift: '},
                    'steps': [{
                        'args': [[frame.name], {
                            'frame': {'duration': 300, 'redraw': True},
                            'mode': 'immediate'
                        }],
                        'label': frame.name,
                        'method': 'animate'
                    } for frame in frames]
                }],
                width=1000,
                height=800,
                template='plotly_white'
            )
            
            fig.frames = frames
            
            # Export frames if save_dir exists
            if self.save_dir:
                self.export_animation_frames(fig, frames, 'redshift_evolution')
                
                # Save HTML
                html_path = os.path.join(self.save_dir, 'standalone_redshift_evolution.html')
                fig.write_html(html_path, include_mathjax='cdn')
                logger.info(f"Saved standalone visualization to {html_path}")
            
            return fig
            
        except Exception as e:
            logger.error(f"Error in standalone redshift visualization: {e}", exc_info=True)
            return None
    
    # [Add this new method to your existing class]
    def save_2d_plots_as_pdf(self, fig, filename):
        """Save 2D plots as PDF with black and white styling."""
        try:
            if not self.save_dir:
                logger.warning("No save directory specified for PDF export")
                return
                
            # Convert to static image first
            img_bytes = fig.to_image(format="png")
            img = Image.open(io.BytesIO(img_bytes))
            
            # Save as PDF
            pdf_path = os.path.join(self.save_dir, filename)
            img.save(pdf_path, "PDF", resolution=100.0)
            logger.info(f"Saved 2D plot as PDF to {pdf_path}")
            
        except Exception as e:
            logger.error(f"Error saving 2D plot as PDF: {e}", exc_info=True)

    # [Modify all methods that create 2D plots to call save_2d_plots_as_pdf]
    # For example, modify density_profile() like this:
    def density_profile(self):
        try:
            r_bins, density_profile = self.__calculate_density_profile()
            fig = self.__plot_density_profile(r_bins, density_profile, 
                                            'Dark Matter Halos Density Profile', 
                                            r'Density (Msun/Mpc^{3})')
            
            if fig and self.save_dir:
                self.save_2d_plots_as_pdf(fig, 'density_profile.pdf')
                
            return fig
        except Exception as e:
            logger.error(f"Error in density profile calculation: {e}")
            return None
    
    # [Similarly modify all other 2D plotting methods]
    
    # [Keep all your other existing methods]
    # ... [rest of your existing code remains unchanged] ...